# Searchligh Analysis
* A classification problem
  * Two conditions: positive / negative
  
### Data description
* Three subjects
  * sub-01, sub-02, sub-03
* Images
  * aligned in MNI space
  * beta-values
* Run in ROI mask
  * Left precentral gyrus in AAL atlas

In [1]:
# initialize data
data_dir = '/home/ubuntu/data/'
result_dir = '/home/ubuntu/results/'
subj_list = ['sub-01', 'sub-02', 'sub-03']
num_subj = len(subj_list)

In [2]:
# initialize headers
import nilearn.decoding
import nilearn.image
import pandas as pd
import time

from sklearn.model_selection import KFold

* If you got import error in 'pandas', run `!pip3 install pandas`

### Check data structure
* `*_bold*.nii.gz`: beta value time-series data
* `*_event*.csv`: behavioral table
* `l_precentral_mask.nii.gz`: the roi mask

In [3]:
!ls $data_dir

l_precentral_mask.nii.gz  sub-02_bold1.nii.gz  sub-03_bold2.nii.gz
sub-01_bold1.nii.gz	  sub-02_bold2.nii.gz  sub-03_event1.csv
sub-01_bold2.nii.gz	  sub-02_event1.csv    sub-03_event2.csv
sub-01_event1.csv	  sub-02_event2.csv
sub-01_event2.csv	  sub-03_bold1.nii.gz


### Check behavioral data
* degree: The class which I want to classify
* order: time index in beta-value time series images

In [4]:
labels = pd.read_csv(data_dir + 'sub-01_event1.csv')
labels.head()

,Unnamed: 0,degree,order
0,0,-1,1
1,1,1,2
2,2,-1,3
3,3,1,4
4,4,1,5


## Run SVC searchlight
* See here: https://nilearn.github.io/decoding/searchlight.html

In [5]:
mask_img = nilearn.image.load_img(data_dir + 'l_precentral_mask.nii.gz')
cv = KFold(n_splits=2)

for subj in subj_list:
    print('running %s ...' % subj, end='')
    
    # initializing searchlight instance
    searchlight = nilearn.decoding.SearchLight(
        mask_img,
        radius=5,
        estimator='svc',
        n_jobs=2,
        verbose=False,
        cv=cv
    )
    
    # loading behavioral data
    label1 = pd.read_csv(data_dir + '%s_event1.csv' % subj)
    label2 = pd.read_csv(data_dir + '%s_event2.csv' % subj)
    
    # loading fMRI images
    run1_img = nilearn.image.load_img(data_dir + '%s_bold1.nii.gz' % subj)
    run2_img = nilearn.image.load_img(data_dir + '%s_bold2.nii.gz' % subj)
    
    # slicing images with order
    run1_img = nilearn.image.index_img(run1_img, label1['order']-1)
    run2_img = nilearn.image.index_img(run2_img, label2['order']-1)
    
    # preparing data
    X = nilearn.image.concat_imgs([run1_img, run2_img])
    y = list(label1['degree']) + list(label2['degree'])
    group = [1 for _ in label1['degree']] + [2 for _ in label2['degree']]
    
    # run searchlight
    searchlight.fit(X, y, group)
    
    # save result image
    result_img = nilearn.image.new_img_like(mask_img, searchlight.scores_)
    result_img.to_filename(result_dir + '%s_result.nii.gz' % subj)
    
    print('done.')

running sub-01 ...done.
running sub-02 ...done.
running sub-03 ...done.


# Comparing results
* Run group analysis with one sample t-test
* See here: https://nilearn.github.io/modules/generated/nilearn.mass_univariate.permuted_ols.html

In [6]:
!ls $result_dir

sub-01_result.nii.gz  sub-02_result.nii.gz  sub-03_result.nii.gz


In [7]:
# initialize header
import numpy as np
import nilearn.mass_univariate

In [8]:
# load data

scores_each_subject = []

for subj in subj_list:
    score = nilearn.image.load_img(result_dir + '%s_result.nii.gz' % subj).get_data()
    scores_each_subject.append(score)
    
scores_each_subject = np.array(scores_each_subject)

In [9]:
# show data summary

shape = scores_each_subject[1, :, :, :].shape

for i, subj in enumerate(subj_list):
    data = scores_each_subject[i, :, :, :].flat
    
    mean = np.mean(data[np.nonzero(data)])
    std = np.std(data[np.nonzero(data)])
    median = np.median(data[np.nonzero(data)])
    print('%s: mean %.2f, median %.2f, std %.2f' % (subj, mean, median, std))
    
    # sub chance level
    scores_each_subject[i, :, :, :] = (scores_each_subject[i, :, :, :] - 0.5) 

sub-01: mean 0.52, median 0.52, std 0.05
sub-02: mean 0.51, median 0.51, std 0.05
sub-03: mean 0.51, median 0.51, std 0.04


In [10]:
# perform statistical test and save t map

t_img = np.zeros(shape)
p_img = np.zeros(shape)

for j in range(shape[0]):
    for k in range(shape[1]):
        for l in range(shape[2]):
            # check voxel is in group mask
            if scores_each_subject[0, j, k, l] != 0:
                # perform permuted OLS
                p_score, t_score, _ = nilearn.mass_univariate.permuted_ols(
                    np.ones((num_subj, 1)),  # one group
                    scores_each_subject[:, j, k, l].reshape(-1, 1),  # make data (num_subject, data vector)
                    n_perm=100,
                    two_sided_test=True,
                    n_jobs=2)
                
                # save results as image
                t_img[j, k, l] = t_score
                p_img[j, k, l] = p_score

        print('%d, %d, ..., ' % (j, k), end='\r')
        
t_img = nilearn.image.new_img_like(mask_img, t_img)
t_img.to_filename(result_dir + 'tstat.nii.gz')

p_img = nilearn.image.new_img_like(mask_img, p_img)
p_img.to_filename(result_dir + 'pstat.nii.gz')

/home/ubuntu/.local/lib/python3.5/site-packages/nilearn/mass_univariate/permuted_least_squares.py:138: RuntimeWarning: invalid value encountered in sqrt
  return beta_targetvars_testedvars * np.sqrt((dof - 1.) / rss)


/home/ubuntu/.local/lib/python3.5/site-packages/nilearn/mass_univariate/permuted_least_squares.py:138: RuntimeWarning: divide by zero encountered in true_divide
  return beta_targetvars_testedvars * np.sqrt((dof - 1.) / rss)
